# RAiDER in Python

RAiDER can be called and used from with Python. 

In [1]:
# at the most basic level, can "import RAiDER"
import RAiDER
import os

In [2]:
# Set up a custom logging location
log_dir = './logs/'
if ~os.path.exists(log_dir):
    os.mkdir(log_dir)
import RAiDER.cli.conf as conf
conf.LOGGER_PATH = log_dir
from RAiDER.logger import logger

In [3]:
# we'll need some other basic libraries
import datetime
import pyproj
import xarray
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

## Downloading weather model data
RAiDER provides access to multiple weather models through a consistent API. 
RAiDER will process the weather model data to a regular 3D cube and write the data to a NETCDF file that will be used later by the `tropo_delay` function to calculate delays at specified query points. 

In order to download weather model data, we need to instantiate a weather model object of the appropriate class. 

In [4]:
# We can print out the list of models currently implemented in RAiDER
# Note that some of these models require licenses etc. 
from RAiDER.models.allowed import ALLOWED_MODELS
print(ALLOWED_MODELS)

['ERA5', 'ERA5T', 'HRRR', 'GMAO', 'HRES']


In [5]:
# The HRRR model from NOAA can be accessed for free
from RAiDER.models.hrrr import HRRR, HRRRAK

To access weather model data, we need only to specify a datetime and a bounding box in South-North-West-East format

In [6]:
# First define datetime, and AOI
date_time = datetime.datetime(2018,11,13, 23, 0, 0)

# bounding box is given in SNWE format
ll_bounds = [36.8, 36.85, -76.15, -76.05]

In [7]:
print(date_time)
print(ll_bounds)

2018-11-13 23:00:00
[36.8, 36.85, -76.15, -76.05]


`prepareWeatherModel` is the function for accessing weather model data. 

In [8]:
from RAiDER.processWM import prepareWeatherModel

In [9]:
help(prepareWeatherModel)

Help on function prepareWeatherModel in module RAiDER.processWM:

prepareWeatherModel(weather_model, time, wmLoc: str = None, ll_bounds: List[float] = None, download_only: bool = False, makePlots: bool = False, force_download: bool = False) -> str
    Parse inputs to download and prepare a weather model grid for interpolation
    
    Args:
        weather_model: WeatherModel   - instantiated weather model object
        time: datetime                - Python datetime to request. Will be rounded to nearest available time
        wmLoc: str                    - file path to which to write weather model file(s)
        ll_bounds: list of float      - bounding box to download in [S, N, W, E] format
        download_only: bool           - False if preprocessing weather model data
        makePlots: bool               - whether to write debug plots
        force_download: bool          - True if you want to download even when the weather model exists
    
    Returns:
        str: filename 

Instantiate the weather model and then pass it to `prepareWeatherModel`

In [10]:
weather_model = HRRR()

In [11]:
# For the first example, we'll do a zenith calculation at the weather model grid nodes
prepareWeatherModel(weather_model, date_time, ll_bounds=ll_bounds, makePlots=True)

Extent of the weather model is (xmin, ymin, xmax, ymax):-97.50, 38.50, -97.50, 38.50
Extent of the input is (xmin, ymin, xmax, ymax): -76.15, 36.80, -76.05, 36.85


'/Users/jlmd9g/software/RAiDER-docs/notebooks/RAiDER_tutorial/weather_files/HRRR_2018_11_13_T23_00_00_37N_38N_77W_76W.nc'

You can look at the PDF files generated to see slices of the weather model variables at different heights. 

We can also load the weather model using xarray:

In [12]:
# we can get the name of the weather model file by passing the write directory to the 'out_file' method
weather_model_file = weather_model.out_file('weather_files')
ds = xarray.load_dataset(weather_model_file)

In [13]:
ds

<xarray.Dataset>
Dimensions:      (x: 14, y: 15, z: 145)
Coordinates:
  * x            (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 9.0 10.0 11.0 12.0 13.0
  * y            (y) float64 0.0 1.0 2.0 3.0 4.0 ... 10.0 11.0 12.0 13.0 14.0
  * z            (z) float32 -500.0 -300.0 -200.0 ... 7.458e+04 8.03e+04
Data variables:
    CRS          |S1 b''
    latitude     (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    longitude    (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    t            (z, y, x) float32 284.1 284.2 284.2 284.2 ... 207.1 207.1 207.1
    p            (z, y, x) float32 1.013e+05 1.013e+05 ... 5.338e+03 5.338e+03
    e            (z, y, x) float32 1.089e+03 1.092e+03 ... 0.06045 0.06052
    wet          (z, y, x) float32 51.49 51.61 51.61 ... 0.005353 0.005359
    hydro        (z, y, x) float32 276.6 276.6 276.5 276.5 ... 20.0 20.0 20.0
    wet_total    (z, y, x) float32 0.2589 0.2592 0.2594 0.2598 ... 0.0 0.0 0.0
    hydro_total  (z, y, x) float32 3.551 3.551 3.55 3.55 ... 0.0 0.0 0.0 0.0
Attributes:
    Conventions:   CF-1.6
    datetime:      2018_11_13T23_00_00
    date_created:  2023_04_19T17_13_20
    title:         Weather model data and delay calculations

In [14]:
# Plot a slice of the total delay at 500 m height
(ds['wet_total'] + ds['hydro_total']).interp(z=500).plot()

"wet_total" and "hydro_total" are the zenith delays (ZTD) at the weather model grid nodes. 

We can now run the delay calculation using various input query points

### Compare to HRRR-AK

We can compare to using HRRR in Alaska. RAiDER can tell when you want to process data in Alaska versus the Continental US (CONUS), and will automatically switch to the HRRR-AK model for a bounding box within the extent of that model. 

In [15]:
# bounding box for south-central AK
date_time = datetime.datetime(2018,11,1, 0, 0, 0)
ll_bounds_ak = [60.5, 61.5, -151, -149]

In [16]:
# instantiate a new weather model
weather_model_ak = HRRRAK()

In [17]:
prepareWeatherModel(weather_model_ak, date_time, ll_bounds=ll_bounds_ak, makePlots=True)

Extent of the weather model is (xmin, ymin, xmax, ymax):-135.00, 90.00, 1.25, 90.00
Extent of the input is (xmin, ymin, xmax, ymax): -151.00, 60.50, -149.00, 61.50


'/Users/jlmd9g/software/RAiDER-docs/notebooks/RAiDER_tutorial/weather_files/HRRR-AK_2018_11_01_T00_00_00_61N_62N_152W_149W.nc'

In [18]:
weather_model_file_ak = weather_model_ak.out_file('weather_files')
ds_ak = xarray.load_dataset(weather_model_file_ak)
ds_ak

<xarray.Dataset>
Dimensions:      (x: 46, y: 48, z: 145)
Coordinates:
  * x            (x) float64 0.0 1.0 2.0 3.0 4.0 ... 41.0 42.0 43.0 44.0 45.0
  * y            (y) float64 0.0 1.0 2.0 3.0 4.0 ... 43.0 44.0 45.0 46.0 47.0
  * z            (z) float32 -500.0 -300.0 -200.0 ... 7.458e+04 8.03e+04
Data variables:
    CRS          |S1 b''
    latitude     (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    longitude    (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    t            (z, y, x) float32 273.1 273.3 273.6 273.8 ... 212.1 212.1 212.1
    p            (z, y, x) float32 1.011e+05 1.011e+05 ... 5.381e+03 5.381e+03
    e            (z, y, x) float32 420.4 418.7 410.8 ... 0.04182 0.04175 0.04168
    wet          (z, y, x) float32 21.5 21.37 20.93 ... 0.003525 0.003519
    hydro        (z, y, x) float32 287.2 286.9 286.6 286.4 ... 19.69 19.69 19.69
    wet_total    (z, y, x) float32 0.05749 0.05801 0.05814 ... 0.0 0.0 0.0
    hydro_total  (z, y, x) float32 3.45 3.449 3.449 3.449 ... 0.0 0.0 0.0 0.0
Attributes:
    Conventions:   CF-1.6
    datetime:      2018_11_01T00_00_00
    date_created:  2023_04_19T17_26_06
    title:         Weather model data and delay calculations

In [19]:
# Plot a slice of the total delay at 500 m height
(ds_ak['wet_total'] + ds_ak['hydro_total']).interp(z=500).plot()

## Delay calculation

The pre-processed weather model already has zenith delays calculated at the grid nodes, but if we want the delays at specific query points we need to use the `tropo_delay` function in RAiDER. 

In [20]:
from RAiDER.delay import tropo_delay

In [21]:
help(tropo_delay)

Help on function tropo_delay in module RAiDER.delay:

tropo_delay(dt, weather_model_file: str, aoi, los, height_levels: List[float] = None, out_proj: Union[int, str] = 4326, cube_spacing_m: int = None, look_dir: str = 'right')
    Calculate integrated delays on query points. Options are:
    1. Zenith delays (ZTD)
    2. Zenith delays projected to the line-of-sight (STD-projected)
    3. Slant delays integrated along the raypath (STD-raytracing)
    
    Args:
        dt: Datetime                - Datetime object for determining when to calculate delays
        weather_model_File: string  - Name of the NETCDF file containing a pre-processed weather model
        aoi: AOI object             - AOI object
        los: LOS object             - LOS object
        height_levels: list         - (optional) list of height levels on which to calculate delays. Only needed for cube generation.
        out_proj: int,str           - (optional) EPSG code for output projection
        look_dir: str   

We will need a few more objects for this delay calculation depending on the type of delays desired. 
The `weather_model_file` is the file path to the NETCDF file generated from `prepareWeatherModel`. 

## AOI objects
The AOI object is is one of several types depending on what is requested. 

In [22]:
# 1. basic data types and input parameters
from RAiDER.llreader import BoundingBox, StationFile # also available:  RasterRDR

In [23]:
aoi = BoundingBox(ll_bounds)
print(aoi.bounds())

[36.8, 36.85, -76.15, -76.05]


We can do much more with the AOI objects then just define a bounding box. 
For example, we can load lat/lon files or files containing GNSS station lists: 

In [24]:
test_aoi = StationFile('data/sample_gnss_list.csv')
print(test_aoi)

Several methods become available upon creation of the object

In [25]:
test_aoi.bounds()

[29.02655998900001, 38.999923395, -122.959307604, -113.010345532]

In [26]:
test_aoi.projection()

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

We can use the `readLL` and `readZ` methods to access the lat/lon and elevation values. If no elevation values are available from the data, a DEM will be downloaded on the fly. 

In [27]:
test_aoi.readLL()

(array([36.4291786 , 37.59498606, 34.1164069 , ..., 37.54305283,
        37.54302561, 37.55816598]),
 array([-120.26497691, -114.75908956, -117.09319198, ..., -122.01594425,
        -122.01589243, -117.49021941]))

In [28]:
test_aoi.readZ()

array([  56.5231551, 1713.2781038,  762.0717087, ...,   -3.5718802,
         -3.5279006, 1924.5517791])

## The LOS object

The LOS object defines the type of delay to calculate; can be zenith delays (ZTD), slant delays (STD) projected from the zenith delays, and slant delays (STD) integrated along the ray path. 

In [29]:
# "Conventional" refers to slant delays by projection
from RAiDER.losreader import Zenith, Conventional, Raytracing

In [30]:
los = Zenith()

In [31]:
los.is_Zenith()

True

If using the `Conventional` or `Raytracing` objects, an orbit file or look vector files should be supplied

In [ ]:
los_ray = Raytracing('data/S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF', time=date_time)

In [ ]:
los_ray.is_Zenith()

In [ ]:
los_ray.ray_trace()

### ZTD calculation

Delay calculation in this case will be for a uniform cube at fixed height levels and horizontal spacing. 

In [34]:
# calculate ZTD 
ds,_ = tropo_delay(date_time, weather_model.out_file('weather_files'), aoi, los, height_levels=[0, 100, 500, 1000], cube_spacing_m=5000)

Output SNWE: [36.75, 36.85, -76.15, -76.0]
Output cube spacing: 0.05


Because we asked for a cube, the delays are returned as a single xarray Dataset. In other cases e.g. for rasters, wet and hydrostatic delays will be returned as two ndarrays. 

We can look at the output of the delay calculation in the Dataset

In [35]:
ds

<xarray.Dataset>
Dimensions:  (z: 4, y: 3, x: 5)
Coordinates:
  * x        (x) float64 -76.15 -76.1 -76.05 -76.0 -75.95
  * y        (y) float64 36.85 36.8 36.75
  * z        (z) int64 0 100 500 1000
Data variables:
    wet      (z, y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    hydro    (z, y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    crs      int64 -2147483647
Attributes:
    Conventions:     CF-1.7
    title:           RAiDER geo cube
    source:          HRRR_2018_11_13_T23_00_00_37N_38N_77W_76W.nc
    history:         2023-04-21 20:19:50.136460 RAiDER
    description:     RAiDER geo cube - zenith
    reference_time:  2018-11-01 00:00:00

We can look at the output shape etc.

In [36]:
print(ds['wet'].shape)
print(ds['wet'].values.mean())
print(ds['wet'].attrs)

(4, 3, 5)
nan
{'units': 'm', 'description': 'wet zenith delay', 'grid_mapping': 'crs'}


Projection information is maintained in the dataset

In [38]:
print(ds.crs)

<xarray.DataArray 'crs' ()>
array(-2147483647)
Attributes:
    crs_wkt:                      GEOGCRS["WGS 84",ENSEMBLE["World Geodetic S...
    semi_major_axis:              6378137.0
    semi_minor_axis:              6356752.314245179
    inverse_flattening:           298.257223563
    reference_ellipsoid_name:     WGS 84
    longitude_of_prime_meridian:  0.0
    prime_meridian_name:          Greenwich
    geographic_crs_name:          WGS 84
    horizontal_datum_name:        World Geodetic System 1984 ensemble
    grid_mapping_name:            latitude_longitude


### Raytracing calculation

In [61]:
ds,_ = tropo_delay(date_time, 'weather_files/HRRR_2018_11_13_T12_00_00_37N_38N_77W_76W.nc', aoi, los_ray, height_levels=[0, 100, 500, 1000])

Output SNWE: [36.78, 36.87, -76.17, -76.05]
Output cube spacing: 0.03
Look direction: right
Processing slice 1 / 4: 0
Processing slice 2 / 4: 100
Processing slice 3 / 4: 500
Processing slice 4 / 4: 1000


In [21]:
ds

<xarray.Dataset>
Dimensions:          (z: 4, y: 4, x: 6)
Coordinates:
  * x                (x) float64 -76.17 -76.14 -76.11 -76.08 -76.05 -76.02
  * y                (y) float64 36.87 36.84 36.81 36.78
  * z                (z) int64 0 100 500 1000
Data variables:
    wet              (z, y, x) float64 0.4313 0.4318 0.4333 ... 0.328 0.3279
    hydro            (z, y, x) float64 3.344 3.44 3.523 ... 2.381 2.565 2.692
    cube_projection  int64 0
Attributes:
    Conventions:     CF-1.7
    title:           RAiDER geo cube
    source:          HRRR_2018_11_13_T12_00_00_37N_38N_77W_76W.nc
    history:         2022-12-11 03:36:17.396004 RAiDER
    description:     RAiDER geo cube - slant - raytracing
    reference_time:  2018-11-13 23:00:00

In [65]:
# Look at the output
print(ds['wet'].shape)
print(ds['hydro'].shape)
print(ds['wet'].values.mean())
print(ds['hydro'].values.mean())
print(ds['wet'].attrs)

(4, 4, 6)
(4, 4, 6)
0.39695660854055426
3.0515329152900406
{'units': 'm', 'description': 'wet slant - raytracing delay', 'grid_mapping': 'cube_projection'}
